In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder,MinMaxScaler,OrdinalEncoder

from sklearn.model_selection import train_test_split


import folium
import plotly.express as px

from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from sklearn.linear_model import LinearRegression

import time

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

import scipy as sp

import os


In [2]:
PATH_FOLDER = os.path.join(os.getcwd(), "")

In [3]:
df=pd.read_csv("feature_eng_data.csv")

In [4]:
df.set_index('Index', inplace=True)

In [5]:
df['pk_partition'] = pd.to_datetime(df['pk_partition'])

In [6]:
df

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,em_acount,country_id,region_code,gender,age,deceased,salary,Financiacion,Ahorro,Cuentas
Index,,,,,,,,,,,,,,,,,,,,,
0,1375586,2018-01-28,2018-01-12,KHL,1.0,2,0,0,0,0,...,1,0,29.0,0,35,0,87218.10,0,0,1
1,1050611,2018-01-28,2015-08-10,KHE,0.0,3,0,0,0,0,...,1,0,13.0,1,23,0,35548.74,0,0,1
2,1050612,2018-01-28,2015-08-10,KHE,0.0,3,0,0,0,0,...,1,0,13.0,1,23,0,122179.11,0,0,1
3,1050613,2018-01-28,2015-08-10,KHD,0.0,3,1,0,0,0,...,0,0,50.0,0,22,0,119775.54,0,1,0
4,1050614,2018-01-28,2015-08-10,KHE,1.0,3,0,0,0,0,...,1,0,50.0,1,23,0,96796.23,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962919,1166765,2019-05-28,2016-08-14,KHE,0.0,3,0,0,0,0,...,1,0,50.0,1,22,0,43912.17,0,0,1
5962920,1166764,2019-05-28,2016-08-14,KHE,0.0,3,0,0,0,0,...,1,0,26.0,1,23,0,23334.99,0,0,1
5962921,1166763,2019-05-28,2016-08-14,KHE,1.0,2,0,0,0,0,...,1,0,50.0,0,47,0,96796.23,0,0,1


In [7]:
df['diff_productos'] = df.groupby('pk_cid')['Financiacion'].diff()

In [8]:
df['diff_productos']

Index
0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
          ... 
5962919    0.0
5962920    0.0
5962921    0.0
5962922    0.0
5962923    NaN
Name: diff_productos, Length: 5936633, dtype: float64

In [9]:
df['compra'] = (df['diff_productos'] >= 1).astype(int)

In [10]:
df['compra'].value_counts()

compra
0    5853268
1      83365
Name: count, dtype: int64

In [11]:
#Comprobamos con un pk_cid que sabemso que tiene mas de un producto y vemos que detecta las compras correctamente

df[df['pk_cid'] == 16502]

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,region_code,gender,age,deceased,salary,Financiacion,Ahorro,Cuentas,diff_productos,compra
Index,,,,,,,,,,,,,,,,,,,,,
2171490,16502,2018-09-28,2018-09-30,KHE,0.0,3,0,0,0,0,...,28.0,0,57,0,126765.57,0,0,0,NaN,0
2943633,16502,2018-10-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,1,0.0,0
2962972,16502,2018-11-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,1,0.0,0
3628237,16502,2018-12-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,1,0.0,0
4028330,16502,2019-01-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,2,0.0,0
4480638,16502,2019-02-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,2,0.0,0
4876038,16502,2019-03-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,2,0.0,0
5134320,16502,2019-04-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,2,0.0,0
5523395,16502,2019-05-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,2,0.0,0


In [12]:
#Llenamos los NaN del primer mes con '0' ya que no tenemos con que comparar 

In [13]:
df['diff_productos'].fillna(value=0, inplace=True)


In [14]:
#Por tanto 'compra' sera nuestro target para la predisposicion de compra de los productos "CUENTAS"

In [15]:
df['entry_date'] = pd.to_datetime(df['entry_date'])
df['pk_partition'] = pd.to_datetime(df['pk_partition'])

In [16]:
df['Antiguedad'] = (df['pk_partition'] - df['entry_date']) / pd.Timedelta(days=30)


In [17]:
df['Antiguedad'] = df['Antiguedad'].round().astype(int)

### TENDECIA DE COMPRA MENSUAL

In [18]:
df_st = df.copy()

In [19]:
df_st['mes'] = df_st['pk_partition'].dt.to_period('M')


In [20]:
serie_temporal = df_st.groupby('mes')['compra'].sum()


In [21]:
serie_temporal

mes
2018-01       0
2018-02    4070
2018-03    4569
2018-04    3990
2018-05    3662
2018-06    4261
2018-07    4548
2018-08    4266
2018-09    5908
2018-10    7001
2018-11    5820
2018-12    5734
2019-01    5391
2019-02    6736
2019-03    6150
2019-04    5529
2019-05    5730
Freq: M, Name: compra, dtype: int32

In [22]:
serie_temporal = serie_temporal.drop(serie_temporal.index[0])

In [23]:
#Intentaremos predecir cuantos productos de CUENTAS venderemos el mes siguiente (JUNIO DE 2019)

df_regresion = serie_temporal.reset_index()
df_regresion['mes'] = df_regresion['mes'].astype(str)  # Convertimos el periodo a string para la regresión lineal

# Dividimos los datos en características (X) y la variable objetivo (y)
X_st = pd.to_numeric(df_regresion['mes'].str.replace('-', ''), errors='coerce').values.reshape(-1, 1)  # Convertimos el periodo a un formato numérico para la regresión lineal
y_st = df_regresion['compra'].values

# Entrenamos el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_st, y_st)

# Predecimos las compras del mes 06 del año 2019
mes_06_2019 = pd.to_numeric('2019-06'.replace('-', ''), errors='coerce').reshape(-1, 1)  # Convertimos el periodo a un formato numérico para la predicción
prediccion_mes_06_2019 = modelo.predict(mes_06_2019)

print("Predicción de compras para el mes 06 del año 2019:", prediccion_mes_06_2019[0])

Predicción de compras para el mes 06 del año 2019: 5993.24768867949


In [24]:
#Predicción de beneficios

Beneficios = prediccion_mes_06_2019[0]*60
Beneficios

#Beneficios estimados para JUNIO 2019 de Productos de FINANCIACION = 359.594 € 

359594.8613207694

### Predicción de acierto del modelo

In [25]:
df

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,gender,age,deceased,salary,Financiacion,Ahorro,Cuentas,diff_productos,compra,Antiguedad
Index,,,,,,,,,,,,,,,,,,,,,
0,1375586,2018-01-28,2018-01-12,KHL,1.0,2,0,0,0,0,...,0,35,0,87218.10,0,0,1,0.0,0,1
1,1050611,2018-01-28,2015-08-10,KHE,0.0,3,0,0,0,0,...,1,23,0,35548.74,0,0,1,0.0,0,30
2,1050612,2018-01-28,2015-08-10,KHE,0.0,3,0,0,0,0,...,1,23,0,122179.11,0,0,1,0.0,0,30
3,1050613,2018-01-28,2015-08-10,KHD,0.0,3,1,0,0,0,...,0,22,0,119775.54,0,1,0,0.0,0,30
4,1050614,2018-01-28,2015-08-10,KHE,1.0,3,0,0,0,0,...,1,23,0,96796.23,0,0,1,0.0,0,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962919,1166765,2019-05-28,2016-08-14,KHE,0.0,3,0,0,0,0,...,1,22,0,43912.17,0,0,1,0.0,0,34
5962920,1166764,2019-05-28,2016-08-14,KHE,0.0,3,0,0,0,0,...,1,23,0,23334.99,0,0,1,0.0,0,34
5962921,1166763,2019-05-28,2016-08-14,KHE,1.0,2,0,0,0,0,...,0,47,0,96796.23,0,0,1,0.0,0,34


In [26]:
df = df.drop(['entry_date','deceased','diff_productos'], axis=1)


In [27]:
df= pd.get_dummies(df, columns=['entry_channel'], drop_first=True)

In [28]:
#Generamos una copia para despues en el recomendador

df_rc2 = df.copy()

In [29]:
df = df.drop(['pk_cid','pk_partition'], axis = 1)

In [30]:
X = df.drop('compra', axis=1)  # Features
y = df['compra']  # Target

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [32]:
print("Tamaño del conjunto de entrenamiento (X):", X_train.shape)
print("Tamaño del conjunto de prueba (X):", X_test.shape)
print("Tamaño del conjunto de entrenamiento (y):", y_train.shape)
print("Tamaño del conjunto de prueba (y):", y_test.shape)

Tamaño del conjunto de entrenamiento (X): (4749306, 38)
Tamaño del conjunto de prueba (X): (1187327, 38)
Tamaño del conjunto de entrenamiento (y): (4749306,)
Tamaño del conjunto de prueba (y): (1187327,)


In [33]:
def AUC(model, X, y):
    return roc_auc_score(y, model.predict_proba(X)[:, 1])

def cross_validation(model, X, y):
    Kfold_metric = cross_val_score(model, X, y, cv=4, scoring='roc_auc')
    return Kfold_metric.mean()

In [34]:
label_ = []
metric_train_ = []
metric_valida_ = []

In [35]:
def evaluate_model(label, model, X, y):
 model.fit(X,y)
 metric_train = AUC(model,X,y)
 metric_valida = cross_validation(model,X,y)
 
 label_.append(label)
 metric_train_.append(metric_train)
 metric_valida_.append(metric_valida)
 return pd.DataFrame({'Model': label_,
                'AUC Train': metric_train_,
                'AUC Cross-Valida': metric_valida_
                }).sort_values('AUC Cross-Valida')

evaluate_model(label='XGBoost', 
 model = xgb.XGBClassifier(), 
 X=X_train, y=y_train)

### Recomendador

In [36]:
df_rc=pd.read_csv("clusters_v2.csv",index_col=0)

In [37]:
df_rc2 = df_rc2[df_rc2["pk_partition"] == "2019-05-28"]

In [38]:
df_rc2

,pk_cid,pk_partition,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,...,entry_channel_KHD,entry_channel_KHE,entry_channel_KHF,entry_channel_KHK,entry_channel_KHL,entry_channel_KHM,entry_channel_KHN,entry_channel_KHO,entry_channel_KHQ,entry_channel_RED
Index,,,,,,,,,,,,,,,,,,,,,
5519930,657817,2019-05-28,0.0,3,0,0,0,0,0,0,...,False,True,False,False,False,False,False,False,False,False
5519932,657905,2019-05-28,1.0,1,0,0,0,0,0,1,...,False,False,False,False,False,False,False,False,False,False
5519933,657336,2019-05-28,1.0,2,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
5519934,657147,2019-05-28,0.0,2,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
5519935,658184,2019-05-28,1.0,2,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962919,1166765,2019-05-28,0.0,3,0,0,0,0,0,0,...,False,True,False,False,False,False,False,False,False,False
5962920,1166764,2019-05-28,0.0,3,0,0,0,0,0,0,...,False,True,False,False,False,False,False,False,False,False
5962921,1166763,2019-05-28,1.0,2,0,0,0,0,0,0,...,False,True,False,False,False,False,False,False,False,False


In [39]:
df_rc2 = df_rc2[["pk_cid","salary","age","segment","gender","region_code"]]

In [40]:
df_rc3 = df_rc2.merge(df_rc["cluster"], left_on="pk_cid", right_index=True, how="inner")


In [41]:
df_rc3 = df_rc3.set_index("pk_cid")

In [42]:
df_cluster_4 = df_rc3[df_rc3["cluster"] == 4].copy()

In [43]:
df_cluster_4

,salary,age,segment,gender,region_code,cluster
pk_cid,,,,,,
658184,183036.72,42,2,1,28.0,4
656415,143418.03,32,2,1,28.0,4
650478,90712.98,86,2,0,28.0,4
654801,67164.33,63,1,0,28.0,4
652139,34699.20,85,2,0,35.0,4
...,...,...,...,...,...,...
1167084,180860.97,33,2,1,28.0,4
1167081,52423.38,35,2,0,28.0,4
1166671,148876.17,34,2,0,46.0,4


In [44]:
# eliminamos la columna de cluster del dataset para que no se use en el cálculo de la similitud de coseno.
df_cluster_4.drop("cluster", axis = 1, inplace = True)

In [45]:
EXECUTE = True

In [46]:
if EXECUTE:
    # normalizamos nuestro dataset
    st = time.time()
    
    scaler = MinMaxScaler()
    df_normalized = scaler.fit_transform(df_cluster_4)
    df_normalized = pd.DataFrame(df_normalized, columns=df_cluster_4.columns)
    df_normalized.index = df_cluster_4.index


    et = time.time()
    print("Normalization took {} minutes".format(round((et - st)/60, 2)))

    sparse_ratings = sp.sparse.csr_matrix(df_normalized.values)

    st = time.time()
    print("Working with user similarity")
    user_similarity = cosine_similarity(sparse_ratings)
    user_sim_df = pd.DataFrame(user_similarity, index = df_normalized.index, columns = df_normalized.index)
    user_sim_df.columns = map(str, user_sim_df.columns)
    user_sim_df.to_parquet("cf_user_similarity.parquet.gzip")

    print("Working with item similarity")
    item_similarity = cosine_similarity(sparse_ratings.T)
    item_sim_df = pd.DataFrame(item_similarity, index = df_normalized.columns, columns = df_normalized.columns)
    item_sim_df.columns = map(str, item_sim_df.columns)
    item_sim_df.to_parquet("cf_item_similarity.parquet.gzip")

    et = time.time()
    print("Total time to calculate similarity took {} minutes.".format(round((et - st)/60, 2)))

else:
    user_sim_df = pd.read_parquet(os.path.join(PATH_FOLDER, "cf_user_similarity.parquet.gzip"))
    item_sim_df = pd.read_parquet(os.path.join(PATH_FOLDER, "cf_item_similarity.parquet.gzip"))

Normalization took 0.0 minutes
Working with user similarity
Working with item similarity
Total time to calculate similarity took 0.31 minutes.


In [47]:
user_sim_df.head(5)

,658184,656415,650478,654801,652139,651732,652704,673382,672927,674237,...,1167063,1167058,1167038,1167102,1167088,1167084,1167081,1166671,1166712,1166666
pk_cid,,,,,,,,,,,,,,,,,,,,,
658184,1.000000,0.991347,0.605982,0.533564,0.528968,0.612202,0.646346,0.911874,0.657211,0.538790,...,0.987226,0.919884,0.943520,0.693030,0.961371,0.997251,0.636652,0.699310,0.933655,0.595728
656415,0.991347,1.000000,0.547276,0.471300,0.493522,0.559954,0.627747,0.896889,0.630342,0.457248,...,0.991835,0.900750,0.965888,0.629593,0.956688,0.994785,0.630070,0.672245,0.959956,0.606884
650478,0.605982,0.547276,1.000000,0.883837,0.969904,0.997878,0.895442,0.570571,0.929422,0.898465,...,0.613737,0.636694,0.594043,0.707466,0.601648,0.559146,0.841744,0.783948,0.583252,0.740604
654801,0.533564,0.471300,0.883837,1.000000,0.876771,0.875666,0.801043,0.642448,0.791129,0.977750,...,0.532362,0.488379,0.570180,0.489667,0.426743,0.488524,0.697919,0.756723,0.559012,0.555957
652139,0.528968,0.493522,0.969904,0.876771,1.000000,0.976890,0.908611,0.525736,0.925003,0.850568,...,0.569518,0.558123,0.595529,0.553459,0.513691,0.483685,0.857756,0.750958,0.592427,0.767658


In [48]:
item_sim_df.head(5)

,salary,age,segment,gender,region_code
salary,1.000000,0.820187,0.815094,0.671635,0.807368
age,0.820187,1.000000,0.736784,0.668236,0.810894
segment,0.815094,0.736784,1.000000,0.670750,0.824495
gender,0.671635,0.668236,0.670750,1.000000,0.679012
region_code,0.807368,0.810894,0.824495,0.679012,1.000000


In [62]:
def top_users(user, df):
    '''
    This function prints the top 10 similar users based on cosine similarity.
    '''

    df.columns = map(int, df.columns)

    if user not in df.columns:
        return('No data available on user {}'.format(user))

    print('Most Similar Users:\n')

    sim_users = df.sort_values(by = user, ascending=False).index[1:10001]
    sim_values = df.sort_values(by = user, ascending=False).loc[:,user].tolist()[1:10001]

    for user, sim in zip(sim_users, sim_values):
        print('User #{0}, Similarity value: {1:.2f}'.format(user, sim))

    return sim_users, sim_values

In [63]:
user1 = np.random.choice(user_sim_df.index)
user1

1013665

In [64]:
[similar_users, similar_values] = top_users(user1, user_sim_df)

Most Similar Users:

User #1252891, Similarity value: 1.00
User #1062474, Similarity value: 1.00
User #1111443, Similarity value: 1.00
User #1346971, Similarity value: 1.00
User #1137394, Similarity value: 1.00
User #1204163, Similarity value: 1.00
User #1368462, Similarity value: 1.00
User #1387430, Similarity value: 1.00
User #1118726, Similarity value: 1.00
User #1099769, Similarity value: 1.00
User #1383996, Similarity value: 1.00
User #1003439, Similarity value: 1.00
User #1400021, Similarity value: 1.00
User #1012492, Similarity value: 1.00
User #1009381, Similarity value: 1.00
User #1217478, Similarity value: 1.00
User #1373926, Similarity value: 1.00
User #1229438, Similarity value: 1.00
User #1339754, Similarity value: 1.00
User #1190199, Similarity value: 1.00
User #1121501, Similarity value: 1.00
User #1021525, Similarity value: 1.00
User #1132792, Similarity value: 1.00
User #1175965, Similarity value: 1.00
User #1005295, Similarity value: 1.00
User #971058, Similarity valu

In [65]:
similar_values

[0.9998015880497657,
 0.9998015880497657,
 0.9997569893795473,
 0.9997408957442935,
 0.9997098504479416,
 0.9996272276963531,
 0.9995636007719111,
 0.9995636007719111,
 0.9995636007719111,
 0.9995219959217807,
 0.999454001460682,
 0.9994130561168574,
 0.9993501527930875,
 0.9993501527930875,
 0.9993501527930875,
 0.999316251829333,
 0.999316251829333,
 0.9992256122323429,
 0.9992222542156705,
 0.9992133960447536,
 0.9991743510552533,
 0.9991734140264021,
 0.9991014145101726,
 0.9990382460702625,
 0.999027505983459,
 0.9990250427518885,
 0.9990142309951404,
 0.9990121641457282,
 0.9989393435796676,
 0.9989279916146658,
 0.9988683971177076,
 0.9988587654684578,
 0.9988405514662628,
 0.9987324275046039,
 0.9987324275046039,
 0.9987285630317496,
 0.9987219811275836,
 0.9987139329250154,
 0.9987134897211911,
 0.9987013176558381,
 0.9987013176558381,
 0.9986882461454053,
 0.9986768216334407,
 0.9986500910368822,
 0.9986415267627033,
 0.9986317933051954,
 0.998630205932471,
 0.998615333794680

In [68]:
media = np.mean(similar_values[:3300])
media

0.9727629918715042